<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/cc-logo-square.png" width="200" alt="cognitiveclass.ai logo" />

<h1>Monte Carlo Exploring Starts: Black Jack</h1> 

<h2>Table of Contents</h2>
<p>In this lab, you will use Perform Exploring Starts on the BlackJack environment on open AI gym </p>

<ul>
    <li><a href="#UF">Utility Functions and Imports</a></li>
    <li><a href="#F">Monte Carlo Exploring Starts Function</a></li>
    <li><a href="#FE">Monte Carlo Prediction Experiments </a></li>
</ul>
<p>Estimated Time Needed: <strong>25 min</strong></p>

<hr>

In [ ]:
#!conda install -c anaconda seaborn
 
#!pip install gym

<h2 id="UF">Utility Functions and Imports </h2> 

In [ ]:
import gym
import matplotlib.pyplot as plt
import math 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import copy
from collections import defaultdict

In [ ]:
from blackjackutility import get_total,game_result

this function is used to plot the value function

In [ ]:
def plot_value_function(V):
    """
    plot the estimated value function for blackjack 
    Returns:  void plots value function 
    Args:
    V: a dictionary of estimated values for blackjack 
    """
    #range of player score  
    player=[state[0]  for state in  V.keys()]
    max_player=max(player)
    min_player=min(player)
    player_range=np.arange(min_player, 22, 1)
    #range of dealer score      
    dealer=[state[1]  for state in  V.keys()]
    max_dealer=max(dealer)
    min_dealer=min(dealer)
    dealer_range=np.arange(min_dealer, 11, 1)
    #empty array for the value function, first access in the players score second  is the dealer, third is if  there  is an ace    
    V_plot=np.zeros((21-min_player+1,max_dealer-min_dealer+1,2))
    #create a mesh grid for plotting 
    X,Y = np.meshgrid(dealer_range,player_range)

    #populate an array  of values for different  scores not including losing scores 
    for (player,dealer,ace),v in V.items():
        if player<=21 and dealer<=21:
            V_plot[player-min_player,dealer-min_dealer,(1*ace)]=V[(player,dealer,ace)]

    #plot surface        
    fig, ax = plt.subplots(nrows=1, ncols=2, subplot_kw={'projection': '3d'})
    ax[0].plot_wireframe(X,Y, V_plot[:,:,0])
    ax[0].set_title('no ace')
    ax[0].set_xlabel('dealer')
    ax[0].set_ylabel('player ')
    ax[0].set_zlabel('value function')
    ax[1].plot_wireframe(X,Y, V_plot[:,:,1])
    ax[1].set_title('no ace')
    ax[1].set_xlabel('dealer')
    ax[1].set_ylabel('player ')
    ax[1].set_zlabel('value function')
    ax[1].set_title(' ace')
    fig.tight_layout()
    plt.show()

    #plot top view of the surface     
    fig, ax = plt.subplots(nrows=1, ncols=2)   
    ax[0].imshow((V_plot[:,:,0]),extent =[1,10,21,4])
    ax[0].set_title('no ace')
    ax[0].set_xlabel('dealer')
    ax[0].set_ylabel('player ')   
    im=ax[1].imshow(V_plot[:,:,1],extent =[1,10,21,4])
    ax[1].set_title('ace')
    ax[1].set_xlabel('dealer')
    fig.colorbar(im, ax=ax[1])

this function will plot blackjack policy 

In [ ]:
def plot_policy_blackjack(policy):
    """
    plot the policy for blackjack 
    Returns:  void plots policy function 
    Args:
    policy: a dictionary of estimated values for blackjack 
    """    
    #range of player score 
    player=[state[0]  for state in  policy.keys()]
    max_player=max(player)
    min_player=min(player)
    #this vale is use in RL book 
    #min_player=12
    player_range=np.arange(min_player, 22, 1)
    #range of dealer score      
    dealer=[state[1]  for state in policy.keys()]
    max_dealer=max(dealer)
    min_dealer=min(dealer)
    dealer_range=np.arange(min_dealer, 11, 1)
    #empty array for the value function, first access in the players score second  is the dealer, third is if  there  is an ace    
    policy_plot=np.zeros((21-min_player+1,max_dealer-min_dealer+1,2))
    #create a mesh grid for plotting 
    X,Y = np.meshgrid(dealer_range,player_range)
    
    
    #populate an array  of values for different  policy not including losing states above 21 
    for (player,dealer,ace),v in policy.items():
        if player<=21 and dealer<=10 and player>=min_player:
            policy_plot[player-min_player,dealer-min_dealer,(1*ace)]=policy[(player,dealer,ace)]

    
    fig, ax = plt.subplots(nrows=1, ncols=2)   
    ax[0].imshow((policy_plot[:,:,0]),cmap=plt.get_cmap('GnBu', 2),extent =[1,10,21,4])
    ax[0].set_title('no ace')
    ax[0].set_xlabel('dealer')
    ax[0].set_ylabel('player ') 
    

    ax[1].set_title('ace')
    ax[1].set_xlabel('dealer')
    im=ax[1].imshow(policy_plot[:,:,1],extent =[1,10,21,4],cmap=plt.get_cmap('GnBu', 2))
    fig.colorbar(im, ax=ax[1],ticks=[0 , 1])

This function calculates the average number of wins for a game of blackjack given a policy.

In [ ]:
def average_wins(environment,policy=None,episodes=10):
    """
    This function calculates the average number of wins for a game of blackjack given a policy.
    If no policy is provided a random policy is selected.
    Returns: average_wins: the average number of wins 
    std_wins: the average number of wins 
    Args:
    environment:AI gym balckjack envorment object 
    policy:policy for blackjack if none a random  action will be selected 
    episodes: number of episodes 
    """

    win_loss=np.zeros(episodes)

    for episode in range(episodes):
        state=environment.reset()
        done=False

        while (not(done)):
            if policy and isinstance(policy[state],np.int64):
                 
                action=policy[state]
                
            else:
                action=environment.action_space.sample()

            state,reward,done,info=environment.step(action)
        result=game_result(environment,state,show=False)
        if reward==1:
            win_loss[episode]=1
        else:
            win_loss[episode]=0  

        
    average_wins=win_loss.mean()
    std_win=win_loss.std()/np.sqrt(episodes)

    return average_wins ,std_win

 <h2 id="F">Exploring Starts Function </h2> 

This function completes exploring starts and returns the policy, value function and action function. The input parameters are the discount factor, the number of episodes and the open AI gym objects.  The class also Specifies if first-visit  and every-visit method. 

In [ ]:
def monte_carlo_ES( environment,N_episodes=100000, discount_factor=1,first_visit=True,theta=0.0001):
    """
    plot the policy for blackjack 
    Returns:  
    policy: a dictionary of estimated policy for blackjack 
    V: a dictionary of estimated values for blackjack 
    Q: a dictionary of estimated action function
    theta:stoping threshold 
    Args:
    environment:AI gym balckjack envorment object 
    N_episodes:number of episodes 
    discount_factor:discount factor
    first_visit: select first-visit MC (Ture) and every-visit MC (False)
    DELTA: list of deltas for each episode 
    """  
    #a dictionary of estimated values for blackjack 
    V=defaultdict(float)
    #a dictionary of estimated action function for blackjack
    Q=defaultdict(float)
    # number of visits to the action function 
    NumberVisitsValue= defaultdict(float)
    # visits to action function
    NumberVisits= defaultdict(float)
    #dictionary  for policy 
    policy=defaultdict(float) 
    #number  of actions 
    number_actions=environment.action_space.n
    #list of max difference between  value functions per  iteration 
    DELTA=[]

    for i in range(N_episodes):
        #max difference between  value functions
        delta=0
        #list that stores each state and reward for each episode     
        episode=[]
        # reset the  environment for the next episode and find first state  
        state=environment.reset()   
        #random action for the first state 
        action=np.random.randint(2)
        #reward for the first state (only for frozen lake) 
        reward=0.0
        #flag for end of episodes  
        done=False
        #action for the first state 
        action=np.random.randint(number_actions)
        #append firt state, reward and action
        episode.append({'state':state , 'reward':reward,'action':action})
        #Past states for signal visit  Monte Carlo 
        state_action=[(state,action)]
        #enumerate for each episode 
        while not(done):

                #take action and find next state, reward and check if the episode is  done (True)
                (state, reward, done, prob) = environment.step(action)

                #check if a policy for the state exists  
                if isinstance(policy[state],np.int64):
                    #obtain action from policy
                    action=int(policy[state])

                else:
                     #if no policy for the state exists  select a random  action  
                    action=np.random.randint(number_actions)
                #add state reward and action to list 
                episode.append({'state':state , 'reward':reward,'action':action})
                #add  states action this is for fist visit only 
                state_action.append((state,action))
         #reverse list as the return is calculated from the last state
        episode.reverse()
        #append the state-action pairs to a list 
        state_action.reverse()


        #determine the return
        G=0

        for t,step in enumerate(episode):

                #check flag for first visit
                G=discount_factor*G+step['reward']
                #check flag for first visit
                if first_visit:
                    #check if the state has been visited before 
                    if (step['state'],step['action']) not in set(state_action[t+1:]): 

                        #increment counter for action 
                        NumberVisits[step['state'],step['action']]+=1
                        #increment counter for value function 
                        NumberVisitsValue[step['state']]+=1
                        #if the action function value  does not exist, create an array  to store them 
                        if not(isinstance(Q[step['state']],np.ndarray) ):
                            Q[step['state']]= np.zeros((number_actions))

                        #calculate mean of action function Q Value functions V using the  recursive definition of mean 
                        Q[step['state']][step['action']]=Q[step['state']][step['action']]+(NumberVisits[step['state'],step['action']]**-1)*(G-Q[step['state']][step['action']])
                        
                        # record the old value of the value function 

                        v=V[step['state']]
                        
                        V[step['state']]=V[step['state']]+(NumberVisitsValue[step['state']]**-1)*(G-V[step['state']])
                        #update the policy to select the action fuciton argment with the largest value 
                        policy[step['state']]=np.random.choice(np.where(Q[step['state']]==Q[step['state']].max())[0])
                        #find max difference between all value functions per  iteration 
                        delta=max(delta,abs(v-V[step['state']]))


                else:
                         #increment counter for action 
                        NumberVisits[step['state'],step['action']]+=1
                        #increment counter for value function 
                        NumberVisitsValue[step['state']]+=1
                        #if the action function value  does not exist, create an array  to store them 
                        if not(isinstance(Q[step['state']],np.ndarray) ):
                            Q[step['state']]= np.zeros((number_actions))

                        #calculate mean of action function Q Value functions V using the  recursive definition of mean 
                        Q[step['state']][step['action']]=Q[step['state']][step['action']]+(NumberVisits[step['state'],step['action']]**-1)*(G-Q[step['state']][step['action']])
                        v=V[step['state']]
                        V[step['state']]=V[step['state']]+(NumberVisitsValue[step['state']]**-1)*(G-V[step['state']])
                        ##update the policy to select the action fuciton argment with the largest value 
                        policy[step['state']]=np.random.choice(np.where(Q[step['state']]==Q[step['state']].max())[0])
                        #find max difference between all value functions per  iteration 
                        delta=max(delta,abs(v-V[step['state']]))
            
        DELTA.append(delta)
        if delta<theta:
            break

    return policy, V, Q,DELTA

 <h2 id="FE">Monte Carlo Prediction Experiments  </h2> 

Let’s  run different iterations of exploring starts and see how different  parameters such as number of episodes   and discount factor effects performance.

We create an openAI gym blackjack enviroment:

In [ ]:
environment= gym.make('Blackjack-v0')

Let's see what happens if we use a random policy:

In [ ]:
average ,std_win=average_wins(environment,episodes=10000)
print("average number of wins", average)

 Approximately 28 % of the game are won 

Lets perform exploring starts with 1000 episodes :

In [ ]:
 policy, V, Q,DELTA= monte_carlo_ES( environment,N_episodes=1000, discount_factor=1,first_visit=True,theta=0)  

plot delta for each  episode  

In [ ]:
plt.plot(DELTA)
plt.xlabel("episodes")
plt.ylabel("delt ")
plt.show()

We show the optimal policy for blackjack found by Monte Carlo exploring starts; blue represents the white represents a stick:

In [ ]:
plot_policy_blackjack(policy)

It's difficult to determine a policy; this is because we did not specify the appropriate number of episodes. We can plot out the action function:

In [ ]:
plot_value_function(V)

We see a  general trend, as the score of the player increases the value function takes on higher values.  Let see the average result of playing one ten thousand games.

In [ ]:
average ,std_win=average_wins(environment,policy,episodes=10000)
print("average wins:",average,std_win )

An almost 10% improvement from a random policy. Now Lets perform exploring starts with 500000 episodes :

In [ ]:
policy, V, Q,DELTA = monte_carlo_ES( environment,N_episodes=500000, discount_factor=1,first_visit=True,theta=0)  

plot delta for each  episode  

In [ ]:
plt.plot(DELTA)
plt.xlabel("episodes")
plt.ylabel("delt ")
plt.show()

The optimal policy for blackjack. If the agent has no ace, the higher the dealer is showing, the more likely the agent is to hit, the exception is if the dealer has an ace. If the agent has an ace, the strategy is different. The agent will stick if the sum of their cards is over 11 and, for the most part, hold the player's sum is over 18. 

In [ ]:
plot_policy_blackjack(policy)

we plot the value function,  as the agent or player score  reaches 21 or the dealer score is smaller the larger the value becomes.

In [ ]:
plot_value_function(V)

We see as the accuracy is now approximately 43% a 15 % improvement from a random policy:

In [ ]:
average ,std_win=average_wins(environment,policy,episodes=10000)
print("average wins:",average,std_win )

Let's see how the number of episodes change the number of wins.     

In [ ]:
accuracy = [] 
episodes = []

#policy, V, Q, DELTA = monte_carlo_ES( environment,N_episodes=500000, discount_factor=1,first_visit=True,theta=0)

for n_episode in [1,50,100,500,1000,5000,10000,50000,100000,500000]:
    print("n_episode: ", str(n_episode))
    policy, V, Q, DELTA = monte_carlo_ES( environment,N_episodes=n_episode, discount_factor=1,first_visit=True, theta = 0)  
    average ,std_win = average_wins(environment,policy,episodes=10000)
    print("n_episode: ", str(n_episode), " average: ", str(average))
    accuracy.append(average)
    episodes.append(n_episode)

we see that over 1000000 episodes the effect is negligible.

In [ ]:
plt.plot(episodes,accuracy)
plt.title("Number of wins vs number of episodes ")
plt.ylabel('percentage of wins')
plt.xlabel('number of episodes ')
plt.show()

Discount factor changes the number of wins.  

In [ ]:
accuracy=[] 
discounts=[]

for discount in [0,0.01,0.1,0.5,1.0]:
    policy, V, Q, delta = monte_carlo_ES( environment,N_episodes=100000, discount_factor=discount,first_visit=True, theta=0)  
    average ,std_win=average_wins(environment,policy,episodes=10000)
    print("discount: ", str(discount), " average: ", str(average))
    discounts.append(discount)
    accuracy.append(average)

In [ ]:

plt.plot(discounts,accuracy)
plt.title("percentage of wins vs discount factor ")
plt.ylabel('percentage of wins')
plt.xlabel('discount factor')
plt.show()

We see as the Discount factor  increases as the percentage  of wins increase,  the rate of increase begins to slow down when the discount  value is  0.6.   

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question: </h1>

<b> 
   
 Use the function  <code>monte_carlo_ES Monte Carlo Exploring Starts using every-visit MC, use <code>500000</code> discount factor of 1,  Determine the number of wins  after 1000 episodes.
?</b>
</div>

Double-click <b>here</b> for the solution.

<!-- The answer is below:

policy, V, Q= monte_carlo_ES( environment,N_episodes=500000, discount_factor=1,first_visit=False) 
plot_policy_blackjack(policy)
plot_value_function(V)
average ,std_win=average_wins(environment,policy,episodes=10000)
print("average wins:",average )

-->

<b>About the Authors:</b> 

<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.

<b>References</b>

<ol type="1">
    <li>Sutton, Richard S., and Andrew G. Barto. "Reinforcement learning: An introduction[</li>
    <li><a href="https://github.com/openai/gym/blob/master/gym/envs/toy_text/blackjack.py">blackjack openai Gym</a></li>
   

</ol>

Copyright &copy; 2020 <a href="cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu">cognitiveclass.ai</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/">MIT License</a>.